# Convert input data (Jonathan D. Müller)

Loads the raw data of branch or soil chambers and splits files into daily files in monthly folders

In [ ]:
import os
import glob
import re
import pandas as pd
import numpy as np

In [ ]:
# Initialization parameters

# Data input
project_path =  './'

project_path_laser = project_path + '01_rawdata/laser computer/'
project_path_tc    = project_path + '01_rawdata/thermocouples/'
project_path_irga  = project_path + '01_rawdata/irga/'
project_path_par   = project_path + '01_rawdata/PAR/'
project_path_flow  = project_path + '01_rawdata/flow/'

# Output
project_path_output = project_path + '02_preprocessed_data/'

# List of months to process
# - If empty, all available data is processed
# - Otherwise, specify using a string of year-month, e.g. ['2018-06'] or ['2017-03','2018-06']
month_list = ['2022-01']

# Minimum IRGA pump flow
# All gas data from the branch chamber IRGA data below this flow will be removed
min_irga_flow = 4

# Functions

In [ ]:
# Main functions
#---------------

# Delete all relevant files in a folder.
# - Used to remove 1h and 1min monthly files before running to prevent appending to existing files
def empty_dir(directory):
    files = glob.glob(directory + '*')
    for f in files:
        month_id = f[-10:-6] + '-' + f[-6:-4]
        if(month_id in month_list):
            print('Remove ' + f)
            os.remove(f)
        if(not month_list):
            print('Remove ' + f)
            os.remove(f)
    pass

# Read an Aerodyne laser input file, full file
def read_laser_file(input_fn):
    # Read data
    data = [ ]
    with open(input_fn) as f:
        next(f) # Skip the first line
        for line in f:
            data.append(re.split(r'\s', line.strip(), 9))
    # build the generator        
    for line in data:
        if(len(line) < 9):
            line.append('')
    # first element returned is the columns
    columns = ['timestamp','OCS.1','CO2.1','CO2.2','H2O.1','CO2.3','CO.1','OCS.2','CO2.4']
    # build the data frame
    df = pd.DataFrame(data, columns=columns)
    #for index, row in df.iterrows():
    #    df.loc[df['timestamp'] == row['timestamp'], 'timestamp'] = float(df.loc[df['timestamp'] == row['timestamp'], 'timestamp'])
    df['timestamp'] = pd.to_numeric(df['timestamp'])
    # Now apply normal conversions
    df['timestamp'] = pd.to_datetime(df.timestamp, unit='s', origin=pd.Timestamp('1904-01-01')) - pd.DateOffset(seconds=1) # To convert IGOR-time (i.e. Excel 1904)
    df['dayid'] = df['timestamp'].apply(lambda x:(x.year*10000 + x.month*100 + x.day))
    return(df)

# Read an Aerodyne laser input file, full file
def read_laser_file_by_lines(input_fn, lines):
    # Read data
    data = [ ]
    with open(input_fn) as f:
        next(f) # Skip the first line
        data.append(re.split(r'\s', f.readline().strip(), 9))
    # build the generator        
    for line in data:
        if(len(line) < 9):
            line.append('')
    # first element returned is the columns
    columns = ['timestamp','OCS.1','CO2.1','CO2.2','H2O.1','CO2.3','CO.1','OCS.2','CO2.4']
    # build the data frame
    df = pd.DataFrame(data, columns=columns)
    #for index, row in df.iterrows():
    #    df.loc[df['timestamp'] == row['timestamp'], 'timestamp'] = float(df.loc[df['timestamp'] == row['timestamp'], 'timestamp'])
    df['timestamp'] = pd.to_numeric(df['timestamp'])
    # Now apply normal conversions
    df['timestamp'] = pd.to_datetime(df.timestamp, unit='s', origin=pd.Timestamp('1904-01-01')) - pd.DateOffset(seconds=1) # To convert IGOR-time (i.e. Excel 1904)
    df['dayid'] = df['timestamp'].apply(lambda x:(x.year*10000 + x.month*100 + x.day))
    return(df)

# OLD Read an Aerodyne laser input file, full file
def read_laser_file_old(input_fn):
    df = pd.read_csv(input_fn, sep=' ', skiprows=[0], index_col=False, header=0, names=['timestamp','OCS.1','CO2.1','CO2.2','H2O.1','CO2.3','CO.1','OCS.2','CO2.4'])
    df['timestamp'] = pd.to_datetime(df.timestamp, unit='s', origin=pd.Timestamp('1904-01-01')) - pd.DateOffset(seconds=1) # To convert IGOR-time (i.e. Excel 1904)
    df['dayid'] = df['timestamp'].apply(lambda x:(x.year*10000 + x.month*100 + x.day))
    return(df)

# OLD Read an Aerodyne laser input file, full file
def read_laser_file_by_lines_old(input_fn, lines):
    df = pd.read_csv(input_fn, sep=' ', skiprows=[0], nrows=lines, index_col=False, header=0, names=['timestamp','OCS.1','CO2.1','CO2.2','H2O.1','CO2.3','CO.1','OCS.2','CO2.4'])
    df['timestamp'] = pd.to_datetime(df.timestamp, unit='s', origin=pd.Timestamp('1904-01-01')) - pd.DateOffset(seconds=1) # To convert IGOR-time (i.e. Excel 1904)
    df['dayid'] = df['timestamp'].apply(lambda x:(x.year*10000 + x.month*100 + x.day))
    return(df)

# Read a CR1000 input file, full file
def read_cr1000_file(input_fn):
    #df = pd.read_csv(input_fn,skiprows=[0,2,3,4,5], na_values=["NAN"])
    df = pd.read_csv(input_fn,skiprows=[0,2,3], na_values=["NAN"])
    if(df.columns[0] != 'TIMESTAMP'):
        #df = pd.read_csv(input_fn,skiprows=[0,1,3,4,5,6], na_values=["NAN"])
        df = pd.read_csv(input_fn,skiprows=[0,1,3,4], na_values=["NAN"])
    df.rename(columns={'TIMESTAMP':'timestamp'}, inplace=True)
    df['timestamp'] = pd.to_datetime( df.timestamp, format='%Y-%m-%d %H:%M:%S', utc=True, errors="raise")#errors='coerce')
    df['dayid'] = df['timestamp'].apply(lambda x:(x.year*10000 + x.month*100 + x.day))
    return(df)

# Read a CR1000 input file by lines, only some initial lines
def read_cr1000_file_by_lines(input_fn, lines):
    #df = pd.read_csv(input_fn,skiprows=[0,2,3,4], na_values=["NAN"],nrows=lines)
    df = pd.read_csv(input_fn,skiprows=[0,2,3], na_values=["NAN"],nrows=lines)
    if(df.columns[0] != 'TIMESTAMP'):
        #df = pd.read_csv(input_fn,skiprows=[0,1,3,4,5], na_values=["NAN"],nrows=lines)
        df = pd.read_csv(input_fn,skiprows=[0,1,3,4], na_values=["NAN"],nrows=lines)
    df.rename(columns={'TIMESTAMP':'timestamp'}, inplace=True)
    df['timestamp'] = pd.to_datetime( df.timestamp, format='%Y-%m-%d %H:%M:%S', utc=True, errors="raise")#errors='coerce')
    df['dayid'] = df['timestamp'].apply(lambda x:(x.year*10000 + x.month*100 + x.day))
    return(df)

# Read a CR1000 input file, full file
def read_irga_file(input_fn):
    df = pd.read_csv(input_fn,skiprows=[0,2,3,4,5], na_values=["NAN"])
    if(df.columns[0] != 'TIMESTAMP'):
        df = pd.read_csv(input_fn,skiprows=[0,1,3,4,5,6], na_values=["NAN"])
    df.rename(columns={'TIMESTAMP':'timestamp'}, inplace=True)
    df['timestamp'] = pd.to_datetime( df.timestamp, format='%Y-%m-%d %H:%M:%S', utc=True, errors="raise")#errors='coerce')
    df['dayid'] = df['timestamp'].apply(lambda x:(x.year*10000 + x.month*100 + x.day))
    return(df)

# Read a CR1000 input file by lines
def read_irga_file_by_lines(input_fn, lines):
    df = pd.read_csv(input_fn,skiprows=[0,2,3,4], na_values=["NAN"],nrows=lines)
    if(df.columns[0] != 'TIMESTAMP'):
        df = pd.read_csv(input_fn,skiprows=[0,1,3,4,5], na_values=["NAN"],nrows=lines)
    df.rename(columns={'TIMESTAMP':'timestamp'}, inplace=True)
    df['timestamp'] = pd.to_datetime( df.timestamp, format='%Y-%m-%d %H:%M:%S', utc=True, errors="raise")#errors='coerce')
    df['dayid'] = df['timestamp'].apply(lambda x:(x.year*10000 + x.month*100 + x.day))
    return(df)

def remove_obsolete_irga_data(temp):
    temp = temp.copy()
    
    # Rename PAR & temperature
    temp.rename(columns={'RadKipZonen':'par.ambient.umol_m2_s1'}, inplace=True)
    temp.rename(columns={'Tc(8)':'temp.air.ambient.c'}, inplace=True)
    temp.rename(columns={'H2o_6262_mmol_mol':'conc.h2o.irga.ambient.mmol_mol'}, inplace=True)
    temp.rename(columns={'Co2_6262_micmol_mol':'conc.co2.irga.ambient.umol_mol'}, inplace=True)
    temp.rename(columns={'AirFlow_Amb':'pump.flow.irga.lpm'}, inplace=True)
    temp.rename(columns={'Prees_7000':'P.irga.kPa'}, inplace=True)
    
    # Remove bad data
    temp.loc[temp['pump.flow.irga.lpm'] <= min_irga_flow, 'conc.h2o.irga.ambient.mmol_mol'] = np.nan
    temp.loc[temp['pump.flow.irga.lpm'] <= min_irga_flow, 'conc.co2.irga.ambient.umol_mol'] = np.nan
    
    # Keep only relevant columns
    temp = temp[['timestamp','temp.air.ambient.c','par.ambient.umol_m2_s1','conc.h2o.irga.ambient.mmol_mol','conc.co2.irga.ambient.umol_mol','P.irga.kPa','dayid']]
    
    return(temp)

def fix_tc_names(temp):
    temp = temp.copy()
    temp.columns = [c.replace('TcRafat_Avg(', 'Tc(') for c in list(temp.columns)]
    temp.columns = [c.replace('TcRafat(', 'Tc(') for c in list(temp.columns)]
    # Drop record column
    temp.drop('RECORD', axis=1, inplace=True)
    return(temp)

# Special treatment, adding Yasmin's data from other dataloggers
def add_other_tc_data(temp, temp_add):
    # Now add the columns from the additional files, if they exist
    merged = temp.merge(temp_add, on=['timestamp','dayid'], how='left')
    return(merged)

# Writes output files in full temporal resolution
def write_output_file(out_df, date_idx, out_dir, output_fn):
    out_df = out_df.copy()
    # Drop duplicates
    out_df.drop_duplicates(subset = 'timestamp', inplace=True)
    # Sort, in case it's not yet the case
    out_df.sort_values('timestamp', inplace=True)
    
    # Check if output folders exist. If not, create
    month_dir = str(date_idx)[0:4] + "-" + str(date_idx)[4:6]
    if(not os.path.exists(out_dir + month_dir)): # make directory if it doesn't exist
        os.makedirs(out_dir + month_dir)
    # Create file name
    out_fn = out_dir + month_dir + "/" + output_fn + "_" + str(date_idx) + ".csv"
    #print(str(len(out_df.dayid)), out_fn) # Shows final file size)
    # organise data for output
    temp_df = out_df
    # Before saving, remove the index
    temp_df.drop('dayid', axis=1, inplace=True)
    # Move timestamp column to the front
    col = temp_df.pop('timestamp')
    temp_df.insert(0, col.name, col, allow_duplicates=True)
    # Remove timezone information
    temp_df['timestamp'] = temp_df['timestamp'].dt.tz_localize(None)
    # Write data
    temp_df.to_csv(out_fn, index=False, encoding='utf-8', date_format='%Y-%m-%d %H:%M:%S') # Save file
    
# Creates a daily file for each type
def organise_files(project_path, output_path, filetype, tc_add = False):
    # List all files in the directories
    fn_list = sorted(glob.glob(project_path + '*/*', recursive=True))
    if(filetype == 'laser'):
        fn_list = sorted(glob.glob(project_path + '*/*.str', recursive=True))
    saved = []
    
    # Create output path name
    project_path_output = output_path + project_path.split('/')[-2] + '/'

    # For all files in the directory
    for fn_i, fn in enumerate(fn_list):
        # Only run data in the month list
        current_month = fn.replace(project_path[:-1], "")[1:8]
        if((current_month in month_list) or (len(month_list) == 0)):
            if( (filetype == 'irga') & (int(current_month.replace('-','')) < 202007) ):
                continue
            #display(fn.replace(project_path_chambers[:-1], "")[1:8])
            pass
        else:
            continue
        
        # Debugging message
        if(fn_i % 1 == 0): # % 20 to show every 20th file being loaded
            print( '{:<07}'.format(str(round(fn_i * 100 / len(fn_list), 4))) + "%\t\t" + fn.split('\\')[-2] + '/' + fn.split('\\')[-1]) # Show status
            #print( '{:<07}'.format(str(round(fn_i * 100 / len(fn_list), 4))) + "%\t\t" + fn.replace(project_path + "01_rawdata/", "")) # Show status
    
        # Load the current laser file
        if(filetype == 'laser'):
            df = read_laser_file(fn)
        else:
            df = read_cr1000_file(fn)
            if(filetype == 'irga'):
                df = remove_obsolete_irga_data(df)
                project_path_output = output_path + 'ambient/'
            if(filetype == 'tc'):
                df = fix_tc_names(df)
    
        if (fn_i != len(fn_list)-1):
            # Load next file
            if(filetype == 'laser'):
                df_next = read_laser_file_by_lines(fn_list[fn_i+1], lines=1)
            else:
                df_next = read_cr1000_file_by_lines(fn_list[fn_i+1], lines=1)
            next_day = df_next['dayid'].tolist()[0]
            final_file = False
        else:
            next_day = 0
            final_file = True
  
        # Group by per-day id
        grouped = df.groupby(['dayid'])
        #print("    Days in this file:  ", len(grouped))
    
        # For each group
        for group_i, (this_day, day) in enumerate(grouped):
        
            # If we have saved data and if the day matches append the current
            if (len(saved) > 0):
                if (this_day == saved['dayid'].tolist()[0]):
                    day = pd.concat([saved, day], axis=0, ignore_index=True)
                    #print("    Current line count: ", str(len(day.dayid)))
        
            #If this is the final group in the file and this is not the final file and the first group of the next file is the same day
            if (group_i == len(grouped)-1) and (not final_file) and (this_day == next_day):
                saved = day
                continue  
            else:
                if(tc_add): # Add special data from other dataloggers (from Yasmin Bohak)
                    day = add_other_tc_data(day, tc_additional)
                write_output_file(day, this_day, project_path_output, filetype)
    pass

In [ ]:
# Makes a list of relevant dates and corresponding columns
# (By Yasmin L Bohak)
def make_list_from_tc_logbook(project_path,chambers_sensors_fn, silent=True):
    import datetime
    if(not silent): print('Creating list of relevant dates:')
    #Read logbook tc to make list of lists of [sensor_name, start, end] dates
    dates_thermocouples_1 = []
    dates_thermocouples_2 = []

    tc_log = pd.read_excel(project_path + chambers_sensors_fn, sheet_name='tc') #_additional + chambers_sensors_fn, sheet_name='tc')
    tc_log.drop(['comment'], axis=1, inplace=True)
    tc_log.dropna(subset=['date'], inplace=True)
    tc_log['date']=pd.to_datetime(tc_log['date'].astype(str), format='%Y-%m-%d %H:%M:%S')

    done = []
    cols = tc_log.columns
    if(not silent): print('  ',cols)
    for col in cols:
        if col != 'date':
            tc_log[col]=pd.to_numeric(tc_log[col])

    for i, row in tc_log.iterrows():
        for c in range(1,len(cols)):
            if c in done:
            # If this condition is fulfilled then it means that this column was already
            # fully parsed 
                continue
            # Make lists of date ranges that will need to select from each data source
            if row[c] >= 10:
                mux = row[c]
                # Iterate over rest of file to find end date, if doesn't exist, end date is today
                end_index = i + 1
                append_check = False
                while end_index < tc_log.shape[0]:
                    if(not silent): print('  - End index',end_index)
                    if tc_log.iloc[end_index, c] == mux:
                        end_index += 1
                    else:
                        if(not silent): print('  - Entry added',mux, row[0], tc_log.iloc[end_index, 0])
                        # Add data to correct list (logger 1 or 2)
                        if row[c] < 20:
                            dates_thermocouples_1.append([cols[c], mux, row[0], tc_log.iloc[end_index, 0]])
                        if row[c] >= 20:
                            dates_thermocouples_2.append([cols[c], mux, row[0], tc_log.iloc[end_index, 0]])
                        append_check = True
                        break
                if not append_check:
                    today_ts = datetime.datetime.now()
                    today_ts_rounded = today_ts - pd.Timedelta(microseconds=today_ts.microsecond)
                    today_ts_rounded = pd.to_datetime(today_ts_rounded, format='%Y-%m-%d %H:%M:%S')
                    #add data to correct list (logger 1 or 2)
                    if row[c] < 20:
                        dates_thermocouples_1.append([cols[c], mux, row[0], today_ts_rounded])
                        done.append(c)
                    if row[c] >= 20:
                        dates_thermocouples_2.append([cols[c], mux, row[0], today_ts_rounded])
                        done.append(c)
    if(not silent): print('    Output:', dates_thermocouples_1, dates_thermocouples_2)
    return(dates_thermocouples_1, dates_thermocouples_2)

# (converts the mux data from list to mux name in the raw data)
# this function works on the entire combined data set (output of function entire_data_set)
# (By Yasmin L Bohak)
def filter_mux_dates(df_n_filtered, dates_thermocouples):
    data_set_2 = False
    muxes = []
    # Filter out all mux channels that are not related to the project
    for listy in dates_thermocouples:
        if listy[1] >= 20:
            # When 20+ values, then assume soil data as mux might exist also for non-soil data
            data_set_2 = True
            mux = int(listy[1] - 20)
            if mux not in muxes: 
                muxes.append(mux)
        else:
            mux = int(listy[1] - 10)
            if mux not in muxes: 
                muxes.append(mux)
    # Filter by columns but don't delete date and dayid (based on mux channels)
    muxes_to_filter = []
    muxes_renamed = []
    if not data_set_2: # Meaning that we are dealing with data set 1
        for mux in muxes:
            muxes_to_filter.append('T' + str(mux))
            muxes_renamed.append('Tc(' + str(10+mux) + ')')
    else:
        for mux in muxes:
            muxes_to_filter.append('Tc_soil_Avg('+str(mux)+')')
            muxes_renamed.append('Tc(' + str(20+mux) + ')')
    muxes_to_filter.append('timestamp')
    muxes_to_filter.append('dayid')
    muxes_renamed.append('timestamp')
    muxes_renamed.append('dayid')
    df_filtered = df_n_filtered[muxes_to_filter]
    # Rename columns
    df_filtered.columns = muxes_renamed
    
    return(df_filtered)
    
# This function reads all the files in each data set,
# connects them, filters out duplicate data, unifies the timestamp data format and
# adds the dayid field
# (By Yasmin L Bohak)
def load_entire_data_set(project_path_tc_add, silent=False):
    # Note: If using this function in order to read data from an excel file (not raw .dat file),
    #       the function will assume that the first row of data contains the title and all the rest
    #       of the file contains only data. (not related to files with .csv or .xlsx)
    # For .csv files, they need to have a date-time format: 2022-05-04 hh:mm:ss
    #       If this does not work save file as .xlsx file and then will work regardless of date time format.
    if(not silent): print('Loading all data in', project_path_tc_add)
    fn_list = sorted(glob.glob(project_path_tc_add + '*/*', recursive=True))
    # df for entire additional data set of tcs
    df_data_set = pd.DataFrame()
    # Iterate over the files in the list
    for fn_i, fn in enumerate(fn_list): 
        if(not silent): # Show status
            print( '    {:<07}'.format(str(round(fn_i * 100 / len(fn_list), 4))) + "%\t\t" + fn.split('\\')[-2] + '/' + fn.split('\\')[-1])
        extension = os.path.splitext(fn)[1]
        if extension == '.dat':
            df = read_cr1000_file(fn)        
        elif extension == '.csv':
            df = pd.read_csv(fn, na_values=["NAN"])
        elif extension == '.xlsx':
            df = pd.read_excel(fn, na_values=["NAN"])
        # Add dayid
        df.rename(columns={'TIMESTAMP':'timestamp'}, inplace=True)
        df['timestamp'] = pd.to_datetime(df.timestamp, format='%Y-%m-%d %H:%M:%S', utc=True, errors="raise")#errors='coerce')
        #df['timestamp'] = df['timestamp'].dt.tz_localize(None)
        df['dayid'] = df['timestamp'].apply(lambda x:(x.year*10000 + x.month*100 + x.day))
        df_data_set = pd.concat([df_data_set, df],axis=0)
    before = df_data_set.shape
    # Remove duplicates from final df (same row of data might exist in two files)
    df_data_set.drop_duplicates(subset=None, keep='first', inplace=True)
    after = df_data_set.shape
    removed = before[0] - after[0]
    if removed != 0:
        print('    ', removed,'lines of duplicate data were removed from data set:', project_path_tc_add.split("/")[-2])
    return(df_data_set) 

In [ ]:
# Load additional data from additional Tc dataloggers (Yasmin L Bohak)

# Additional data, logbook etc.
project_path_additional = project_path + '00_additional_data/'
chambers_sensors_fn = 'COS_chambers_sensor_logbook.xlsx'

#paths to the additional Tc data, not in main tc data set
project_path_tc1 = project_path + '01_rawdata/thermocouples_1/'
project_path_tc2 = project_path + '01_rawdata/thermocouples_2/'

# Extract relevant dates
dates_tc1, dates_tc2 = make_list_from_tc_logbook(project_path_additional, chambers_sensors_fn, silent=True)

# Load all data from 2 additional dataloggers
tc1_df = load_entire_data_set(project_path_tc1)
tc2_df = load_entire_data_set(project_path_tc2)

# Filter relevant dates and rename columns
out_tc1 = filter_mux_dates(tc1_df, dates_tc1)
out_tc2 = filter_mux_dates(tc2_df, dates_tc2)

# Merge
tc_additional = out_tc1.merge(out_tc2, on=['timestamp','dayid'], how='outer')

print('Done...')

# Convert files to daily output

In [ ]:
# Run calculations
#-----------------

print('Laser files')
organise_files(project_path_laser, project_path_output, 'laser')

print("Done...")

In [ ]:
print('PAR files')
organise_files(project_path_par, project_path_output, 'par')

print("Done...")

In [ ]:
print('Thermocouple files')
#organise_files(project_path_tc, project_path_output, 'tc') # Normal case
organise_files(project_path_tc, project_path_output, 'tc', tc_add = True)

print("Done...")

In [ ]:
print('Flow rate files')
organise_files(project_path_flow, project_path_output, 'flow')

print("Done...")

In [ ]:
# IRGA files needed for the ambient data. There is a lot, so it's slow...
print('IRGA files')
organise_files(project_path_irga, project_path_output, 'irga')

print("Done...")

# Missing data replacement

There were gaps in the measurements due to device failure. Data was considered to be stable during this time period and was therefore replaced as follows:

- FR gap:    [started at 11th July 2021 end 7th Aug 2021], we will use 10th July 2021 for July days, and 9th Aug 2021 for Aug days  
- Tc gap:    [started at 13th July 2021 end 23rd July 2021], we will use 11th July 2021 for July days
- PAR gap:   [started at 11th July 2021 end 7th Aug 2021], we will use 10th July 2021 for July days, and 8th Aug 2021 for Aug days

In [ ]:
# Replace bad Tc, July 2021

# Read input file
input_fn = project_path_output + 'thermocouples/' + '2021-07/tc_20210711.csv'
print(input_fn)
input_df = pd.read_csv(input_fn)
input_df['timestamp'] = pd.to_datetime(input_df.timestamp, format='%Y-%m-%d %H:%M:%S')

# Calculate number of days offset
days_offset = str(input_df['timestamp'].dt.day.values[0] - 1) + 'days'
current_month = input_df['timestamp'].dt.month.values[0]
input_df['timestamp'] = input_df['timestamp'] - pd.Timedelta(days_offset)

# July
for i in np.arange(14, 23):
    out_df = input_df.copy()
    out_df['timestamp'] = out_df['timestamp'] + pd.Timedelta(str(i-1) + 'day')
    out_df['dayid'] = 2021*10000 + current_month*100 + i
    date_idx = 2021*10000 + current_month*100 + i
    print(date_idx)
    write_output_file(out_df, date_idx, project_path_output + 'thermocouples/', 'tc')
    pass

In [ ]:
# Replace bad flow, July 2021

# Read input file
input_fn = project_path_output + 'flow/' + '2021-07/flow_20210710.csv'
print(input_fn)
input_df = pd.read_csv(input_fn)
input_df['timestamp'] = pd.to_datetime(input_df.timestamp, format='%Y-%m-%d %H:%M:%S')

# Calculate number of days offset
days_offset = str(input_df['timestamp'].dt.day.values[0] - 1) + 'days'
current_month = input_df['timestamp'].dt.month.values[0]
input_df['timestamp'] = input_df['timestamp'] - pd.Timedelta(days_offset)

# July
for i in np.arange(11, 32):
    out_df = input_df.copy()
    out_df['timestamp'] = out_df['timestamp'] + pd.Timedelta(str(i-1) + 'day')
    out_df['dayid'] = 2021*10000 + current_month*100 + i
    date_idx = 2021*10000 + current_month*100 + i
    print(date_idx)
    write_output_file(out_df, date_idx, project_path_output + 'flow/', 'flow')
    pass

# Replace bad flow, Aug 2021

# Read input file
input_fn = project_path_output + 'flow/' + '2021-08/flow_20210809.csv'
print(input_fn)
input_df = pd.read_csv(input_fn)
input_df['timestamp'] = pd.to_datetime(input_df.timestamp, format='%Y-%m-%d %H:%M:%S')

# Calculate number of days offset
days_offset = str(input_df['timestamp'].dt.day.values[0] - 1) + 'days'
current_month = input_df['timestamp'].dt.month.values[0]
input_df['timestamp'] = input_df['timestamp'] - pd.Timedelta(days_offset)

# July
for i in np.arange(1, 8):
    out_df = input_df.copy()
    out_df['timestamp'] = out_df['timestamp'] + pd.Timedelta(str(i-1) + 'day')
    out_df['dayid'] = 2021*10000 + current_month*100 + i
    date_idx = 2021*10000 + current_month*100 + i
    print(date_idx)
    write_output_file(out_df, date_idx, project_path_output + 'flow/', 'flow')
    pass

In [ ]:
# Replace bad PAR, July 2021

# Read input file
input_fn = project_path_output + 'PAR/' + '2021-07/par_20210710.csv'
print(input_fn)
input_df = pd.read_csv(input_fn)
input_df['timestamp'] = pd.to_datetime(input_df.timestamp, format='%Y-%m-%d %H:%M:%S')

# Calculate number of days offset
days_offset = str(input_df['timestamp'].dt.day.values[0] - 1) + 'days'
current_month = input_df['timestamp'].dt.month.values[0]
input_df['timestamp'] = input_df['timestamp'] - pd.Timedelta(days_offset)

# July
for i in np.arange(12, 32):
    out_df = input_df.copy()
    out_df['timestamp'] = out_df['timestamp'] + pd.Timedelta(str(i-1) + 'day')
    out_df['dayid'] = 2021*10000 + current_month*100 + i
    date_idx = 2021*10000 + current_month*100 + i
    print(date_idx)
    write_output_file(out_df, date_idx, project_path_output + 'PAR/', 'par')
    pass

# Replace bad PAR, Aug 2021

# Read input file
input_fn = project_path_output + 'PAR/' + '2021-08/par_20210808.csv'
print(input_fn)
input_df = pd.read_csv(input_fn)
input_df['timestamp'] = pd.to_datetime(input_df.timestamp, format='%Y-%m-%d %H:%M:%S')

# Calculate number of days offset
days_offset = str(input_df['timestamp'].dt.day.values[0] - 1) + 'days'
current_month = input_df['timestamp'].dt.month.values[0]
input_df['timestamp'] = input_df['timestamp'] - pd.Timedelta(days_offset)

# July
for i in np.arange(1, 8):
    out_df = input_df.copy()
    out_df['timestamp'] = out_df['timestamp'] + pd.Timedelta(str(i-1) + 'day')
    out_df['dayid'] = 2021*10000 + current_month*100 + i
    date_idx = 2021*10000 + current_month*100 + i
    print(date_idx)
    #display(out_df)
    write_output_file(out_df, date_idx, project_path_output + 'PAR/', 'par')
    pass